In [ ]:
#prophet

In [1]:
!pip install prophet

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import os
from prophet import Prophet
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import warnings

# -----------------------------
# Suppress warnings
# -----------------------------
warnings.filterwarnings("ignore")

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "prophet_output_models"
output_csv = "prophet_output_csv"
output_metrics_csv = "prophet_metrics.csv"
output_graphs = "prophet_output_graphs"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Prepare metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Function to robustly parse dates (all formats)
# -----------------------------
def parse_dates_safe(date_series):
    return pd.to_datetime(date_series, errors='coerce', dayfirst=False, infer_datetime_format=True)

# -----------------------------
# Function to calculate MAPE safely
# -----------------------------
def mean_absolute_percentage_error_safe(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Avoid division by zero
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    # Handle any remaining inf or NaN
    if np.isnan(mape) or np.isinf(mape):
        return np.nan
    return mape

# -----------------------------
# Process each CSV
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"========== Processing: {file} ==========")

        # Load CSV
        df = pd.read_csv(file_path)

        # -----------------------------
        # Convert 'Date' column safely
        # -----------------------------
        df['Date'] = parse_dates_safe(df['Date'])
        df = df.sort_values('Date')

        # Drop rows with NaT after conversion
        df = df.dropna(subset=['Date'])
        if df.empty:
            print(f"⚠️ No valid dates found in {file}. Skipping...")
            continue

        # -----------------------------
        # Handle missing values in Average Price
        # -----------------------------
        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'].fillna(mean_value, inplace=True)
            print(f"Filled {df['Average Price'].isna().sum()} missing values with mean: {mean_value:.2f}")

        # Round actual values
        df['Average Price'] = df['Average Price'].round(2)

        # -----------------------------
        # Prepare data for Prophet
        # -----------------------------
        prophet_df = df[['Date', 'Average Price']].rename(columns={'Date': 'ds', 'Average Price': 'y'})

        # Skip files with less than 2 valid rows (Prophet requirement)
        if len(prophet_df) < 2:
            print(f"⚠️ Not enough data points in {file} for Prophet. Skipping...")
            continue

        # Build and fit Prophet model
        model = Prophet(daily_seasonality=True)
        model.fit(prophet_df)

        # Predict for existing dates
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)
        df['Predicted'] = forecast['yhat']  # keep full precision for metrics

        # -----------------------------
        # Handle NaNs in predictions for metrics
        # -----------------------------
        metrics_df = df.dropna(subset=['Average Price', 'Predicted'])
        y_true = metrics_df['Average Price'].values
        y_pred = metrics_df['Predicted'].values

        # Round predictions for saving
        df['Predicted'] = df['Predicted'].round(2)

        # Rename Average Price to Actual in main df for plotting
        df.rename(columns={'Average Price': 'Actual'}, inplace=True)

        # -----------------------------
        # Calculate metrics
        # -----------------------------
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)
        mape = round(mean_absolute_percentage_error_safe(y_true, y_pred), 2)
        accuracy = round(100 - mape if not np.isnan(mape) else np.nan, 2)

        print(f"Metrics for {file}:")
        print(f"  MAE        : {mae}")
        print(f"  RMSE       : {rmse}")
        print(f"  R²         : {r2}")
        print(f"  MAPE(%)    : {mape}")
        print(f"  Accuracy(%) : {accuracy}\n")

        # Save metrics
        metrics_list.append({
            'File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # -----------------------------
        # Save model
        # -----------------------------
        model_file = os.path.join(output_models, file.replace(".csv", "_prophet_model.pkl"))
        joblib.dump(model, model_file)

        # -----------------------------
        # Save CSV with only Date, Actual, and Predicted
        # -----------------------------
        save_df = df[['Date', 'Actual', 'Predicted']]
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_prophet_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)
        print(f"Saved updated CSV with Date, Actual & Predicted: {updated_csv_path}")

        # -----------------------------
        # Add MA7 & MA30 for graphs
        # -----------------------------
        df['MA7'] = df['Actual'].rolling(7).mean()
        df['MA30'] = df['Actual'].rolling(30).mean()

        # -----------------------------
        # Plot graph
        # -----------------------------
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['Actual'], label='Actual', color='blue')
        plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red')
        plt.plot(df['Date'], df['MA7'], label='MA7', color='green', linestyle='--')
        plt.plot(df['Date'], df['MA30'], label='MA30', color='orange', linestyle='--')
        plt.title(f"{file} - Actual vs Predicted with Moving Averages")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.legend()
        plt.grid(True)
        graph_path = os.path.join(output_graphs, file.replace(".csv", "_prophet_graph.png"))
        plt.savefig(graph_path, bbox_inches='tight')
        plt.close()

# -----------------------------
# Save all metrics to CSV
# -----------------------------
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)
print(f"✅ Multiple CSVs processed! Metrics saved to {output_metrics_csv}. Models, updated CSVs, and graphs saved in respective folders.")

========== Processing: tomato_Bangalore_daily.csv ==========


08:31:48 - cmdstanpy - INFO - Chain [1] start processing
08:32:33 - cmdstanpy - INFO - Chain [1] done processing


Metrics for tomato_Bangalore_daily.csv:
  MAE        : 738.43
  RMSE       : 1157.81
  R²         : 0.3171
  MAPE(%)    : 52.34
  Accuracy(%) : 47.66

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\tomato_Bangalore_daily_prophet_updated.csv
========== Processing: tomato_Belgaum_daily.csv ==========


08:32:36 - cmdstanpy - INFO - Chain [1] start processing
08:32:39 - cmdstanpy - INFO - Chain [1] done processing


Metrics for tomato_Belgaum_daily.csv:
  MAE        : 629.22
  RMSE       : 1079.74
  R²         : 0.0828
  MAPE(%)    : 72.56
  Accuracy(%) : 27.44

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\tomato_Belgaum_daily_prophet_updated.csv
========== Processing: tomato_Bellary_daily.csv ==========


...

Metrics for tomato_Shimoga_daily.csv:
  MAE        : 712.24
  RMSE       : 1000.97
  R²         : -0.037
  MAPE(%)    : 81.31
  Accuracy(%) : 18.69

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\tomato_Shimoga_daily_prophet_updated.csv
========== Processing: tomato_Tumkur_daily.csv ==========


08:36:14 - cmdstanpy - INFO - Chain [1] start processing
08:36:21 - cmdstanpy - INFO - Chain [1] done processing


Metrics for tomato_Tumkur_daily.csv:
  MAE        : 707.44
  RMSE       : 1227.5
  R²         : -3.0274
  MAPE(%)    : 75.91
  Accuracy(%) : 24.09

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\tomato_Tumkur_daily_prophet_updated.csv
========== Processing: tomato_Udupi_daily.csv ==========


08:36:23 - cmdstanpy - INFO - Chain [1] start processing
08:36:25 - cmdstanpy - INFO - Chain [1] done processing


Metrics for tomato_Udupi_daily.csv:
  MAE        : 658.71
  RMSE       : 880.11
  R²         : 0.0036
  MAPE(%)    : 53.53
  Accuracy(%) : 46.47

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\tomato_Udupi_daily_prophet_updated.csv
✅ Multiple CSVs processed! Metrics saved to prophet_metrics.csv. Models, updated CSVs, and graphs saved in respective folders.


In [ ]:
#ARIMA

In [3]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import warnings

# -----------------------------
# Suppress warnings
# -----------------------------
warnings.filterwarnings("ignore")

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "arima_output_models"
output_csv = "arima_output_csv"
output_metrics_csv = "arima_metrics.csv"
output_graphs = "arima_output_graphs"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Robust date parsing (allow all formats)
# -----------------------------
def parse_dates_safe(date_series):
    return pd.to_datetime(date_series, errors='coerce', infer_datetime_format=True)

# -----------------------------
# Safe MAPE calculation (handle NaN/Inf)
# -----------------------------
def mean_absolute_percentage_error_safe(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return 0.0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    if np.isnan(mape) or np.isinf(mape):
        return 0.0
    return mape

# -----------------------------
# Process each CSV
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"========== Processing: {file} ==========")

        # Load CSV
        df = pd.read_csv(file_path)

        # -----------------------------
        # Convert 'Date' column safely
        # -----------------------------
        df['Date'] = parse_dates_safe(df['Date'])
        df = df.sort_values('Date')

        # Drop rows with invalid dates
        df = df.dropna(subset=['Date'])
        if df.empty:
            print(f"⚠️ No valid dates found in {file}. Skipping...")
            continue

        # -----------------------------
        # Handle missing Average Price
        # -----------------------------
        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'].fillna(mean_value, inplace=True)
            print(f"Filled {df['Average Price'].isna().sum()} missing values with mean: {mean_value:.2f}")

        df['Average Price'] = df['Average Price'].round(2)

        # -----------------------------
        # Fit ARIMA model
        # -----------------------------
        order = (5, 1, 0)
        model = ARIMA(df['Average Price'], order=order)
        model_fit = model.fit()

        # Predict for existing dates
        df['Predicted'] = model_fit.predict(start=0, end=len(df)-1)
        df['Predicted'] = df['Predicted'].round(2)

        # -----------------------------
        # Rename Average Price to Actual
        # -----------------------------
        df.rename(columns={'Average Price': 'Actual'}, inplace=True)

        # -----------------------------
        # Calculate metrics
        # -----------------------------
        y_true = df['Actual'].values
        y_pred = df['Predicted'].values
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)
        mape = round(mean_absolute_percentage_error_safe(y_true, y_pred), 2)
        accuracy = round(100 - mape, 2)

        print(f"Metrics for {file}:")
        print(f"  MAE        : {mae}")
        print(f"  RMSE       : {rmse}")
        print(f"  R²         : {r2}")
        print(f"  MAPE(%)    : {mape}")
        print(f"  Accuracy(%) : {accuracy}\n")

        # Save metrics
        metrics_list.append({
            'File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # -----------------------------
        # Save model
        # -----------------------------
        model_file = os.path.join(output_models, file.replace(".csv", "_arima_model.pkl"))
        joblib.dump(model_fit, model_file)

        # -----------------------------
        # Save CSV with only Date, Actual, Predicted
        # -----------------------------
        save_df = df[['Date', 'Actual', 'Predicted']]
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_arima_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)
        print(f"Saved updated CSV with Date, Actual & Predicted: {updated_csv_path}")

        # -----------------------------
        # Add MA7 & MA30 for graphs
        # -----------------------------
        df['MA7'] = df['Actual'].rolling(7).mean()
        df['MA30'] = df['Actual'].rolling(30).mean()

        # -----------------------------
        # Plot graph
        # -----------------------------
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Actual'], label='Actual', color='blue')
        plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red')
        plt.plot(df['Date'], df['MA7'], label='MA7', color='green', linestyle='--')
        plt.plot(df['Date'], df['MA30'], label='MA30', color='orange', linestyle='--')
        plt.title(f"{file} - Actual vs Predicted with Moving Averages")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.legend()
        plt.grid(True)
        graph_path = os.path.join(output_graphs, file.replace(".csv", "_arima_graph.png"))
        plt.savefig(graph_path, bbox_inches='tight')
        plt.close()

# -----------------------------
# Save all metrics to CSV
# -----------------------------
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)
print(f"✅ Multiple CSVs processed! Metrics saved to {output_metrics_csv}. Models, updated CSVs, and graphs saved in respective folders.")

========== Processing: tomato_Bangalore_daily.csv ==========
Metrics for tomato_Bangalore_daily.csv:
  MAE        : 417.99
  RMSE       : 901.54
  R²         : 0.6118
  MAPE(%)    : 30.57
  Accuracy(%) : 69.43

Saved updated CSV with Date, Actual & Predicted: arima_output_csv\tomato_Bangalore_daily_arima_updated.csv
========== Processing: tomato_Belgaum_daily.csv ==========
Metrics for tomato_Belgaum_daily.csv:
  MAE        : 50.47
  RMSE       : 165.19
  R²         : 0.9784
  MAPE(%)    : 3.45
  Accuracy(%) : 96.55

Saved updated CSV with Date, Actual & Predicted: arima_output_csv\tomato_Belgaum_daily_arima_updated.csv
========== Processing: tomato_Bellary_daily.csv ==========
Metrics for tomato_Bellary_daily.csv:
  MAE        : 117.12
  RMSE       : 221.25
  R²         : 0.873
  MAPE(%)    : 13.45
  Accuracy(%) : 86.55

Saved updated CSV with Date, Actual & Predicted: arima_output_csv\tomato_Bellary_daily_arima_updated.csv
========== Processing: tomato_Chamrajnagar_daily.csv ========

In [ ]:
#SARIMAX

In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import joblib

warnings.filterwarnings("ignore")  # Ignore warnings

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "sarimax_output_models"
output_csv = "sarimax_output_csv"
output_graphs = "sarimax_output_graphs"
output_metrics_csv = "sarimax_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Robust date parsing (allow all formats)
# -----------------------------
def parse_dates_safe(date_series):
    return pd.to_datetime(date_series, errors='coerce', infer_datetime_format=True)

# -----------------------------
# Safe MAPE calculation (handle NaN/Inf)
# -----------------------------
def mean_absolute_percentage_error_safe(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return 0.0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    if np.isnan(mape) or np.isinf(mape):
        return 0.0
    return mape

# -----------------------------
# Process each CSV
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"========== Processing: {file} ==========")

        # Load CSV
        df = pd.read_csv(file_path)

        # -----------------------------
        # Convert 'Date' column safely
        # -----------------------------
        df['Date'] = parse_dates_safe(df['Date'])
        df = df.sort_values('Date')
        df.set_index('Date', inplace=True)

        # Drop rows with invalid dates
        df = df.dropna(subset=['Average Price'])
        if df.empty:
            print(f"⚠️ No valid data found in {file}. Skipping...")
            continue

        # Handle missing values
        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'].fillna(mean_value, inplace=True)
            print(f"Filled {df['Average Price'].isna().sum()} missing values with mean: {mean_value:.2f}")

        # Round actual values
        df['Average Price'] = df['Average Price'].round(2)

        # Add moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()

        # SARIMAX order (tune if needed)
        order = (1, 1, 1)
        seasonal_order = (1, 1, 1, 7)

        # Fit SARIMAX model
        model = SARIMAX(df['Average Price'],
                        order=order,
                        seasonal_order=seasonal_order,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        model_fit = model.fit(disp=False)

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_sarimax_model.pkl"))
        joblib.dump(model_fit, model_file)

        # Predictions
        df['Predicted'] = model_fit.predict(start=0, end=len(df)-1)
        df['Predicted'] = df['Predicted'].round(2)  # round predictions

        # Rename Average Price to Actual
        df.rename(columns={'Average Price': 'Actual'}, inplace=True)

        # Calculate metrics
        y_true = df['Actual'].values
        y_pred = df['Predicted'].values
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)
        mape = round(mean_absolute_percentage_error_safe(y_true, y_pred), 2)
        accuracy = round(100 - mape, 2)

        print(f"Metrics for {file}:")
        print(f"  MAE        : {mae}")
        print(f"  RMSE       : {rmse}")
        print(f"  R²         : {r2}")
        print(f"  MAPE(%)    : {mape}")
        print(f"  Accuracy(%) : {accuracy}\n")

        # Save metrics
        metrics_list.append({
            'File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # -----------------------------
        # Save updated CSV with only Date, Actual, Predicted
        # -----------------------------
        save_df = df[['Actual', 'Predicted']].copy()
        save_df['Date'] = df.index
        save_df = save_df[['Date', 'Actual', 'Predicted']]
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_sarimax_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)
        print(f"Saved updated CSV with Date, Actual & Predicted: {updated_csv_path}")

        # -----------------------------
        # Plot graph
        # -----------------------------
        plt.figure(figsize=(12,6))
        plt.plot(df.index, df['Actual'], label="Actual", color="blue")
        plt.plot(df.index, df['Predicted'], label="Predicted", color="red", linestyle="dashed")
        plt.plot(df.index, df['MA_7'], label="MA 7", color="orange")
        plt.plot(df.index, df['MA_30'], label="MA 30", color="green")
        plt.xlabel("Date")
        plt.ylabel("Average Price")
        plt.title(f"Price Prediction (SARIMAX) - {file}")
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_sarimax_graph.png"))
        plt.savefig(graph_file, bbox_inches='tight')
        plt.close()

# Save all metrics to CSV
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)
print(f"✅ Multiple CSVs processed! Metrics saved to {output_metrics_csv}. Models, updated CSVs, and graphs saved in respective folders.")

========== Processing: tomato_Bangalore_daily.csv ==========
Metrics for tomato_Bangalore_daily.csv:
  MAE        : 417.62
  RMSE       : 877.62
  R²         : 0.6322
  MAPE(%)    : 30.43
  Accuracy(%) : 69.57

Saved updated CSV with Date, Actual & Predicted: sarimax_output_csv\tomato_Bangalore_daily_sarimax_updated.csv
========== Processing: tomato_Belgaum_daily.csv ==========
Metrics for tomato_Belgaum_daily.csv:
  MAE        : 52.25
  RMSE       : 165.63
  R²         : 0.9783
  MAPE(%)    : 3.59
  Accuracy(%) : 96.41

Saved updated CSV with Date, Actual & Predicted: sarimax_output_csv\tomato_Belgaum_daily_sarimax_updated.csv
========== Processing: tomato_Bellary_daily.csv ==========
Metrics for tomato_Bellary_daily.csv:
  MAE        : 119.55
  RMSE       : 222.12
  R²         : 0.872
  MAPE(%)    : 13.93
  Accuracy(%) : 86.07

Saved updated CSV with Date, Actual & Predicted: sarimax_output_csv\tomato_Bellary_daily_sarimax_updated.csv
========== Processing: tomato_Chamrajnagar_daily.

In [ ]:
#TAT+MHA

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import joblib  # For saving models as .pkl

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_output_models"
output_csv = "tat_output_csv"
output_graphs = "tat_output_graphs"
output_logs = "tat_output_logs"
metrics_file = "tat_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Define Temporal Attention Transformer Model
# -----------------------------
def build_tat_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)  # (look_back, 1)
    
    # Multi-Head Attention
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(inputs, inputs)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual connection
    
    # Feed-Forward Network
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual connection
    
    # Global Pooling
    x = layers.GlobalAveragePooling1D()(x)
    
    # Output
    outputs = layers.Dense(1)(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Function to robustly parse dates
# -----------------------------
def parse_dates_safe(date_series):
    # First try strict ISO format (avoids warning)
    try:
        return pd.to_datetime(date_series, format="%Y-%m-%d", errors='coerce')
    except:
        # Fallback flexible parsing
        return pd.to_datetime(date_series, errors='coerce', dayfirst=True)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"🚀 Processing: {file}")

        # Load CSV
        df = pd.read_csv(file_path)
        
        # Robust date parsing
        df['Date'] = parse_dates_safe(df['Date'])
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Replace NaN in 'Average Price' with column mean (future-proof)
        df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())
        df['Average Price'] = df['Average Price'].round(2)

        # Add moving averages (future-proof fill)
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()
        df['MA_7'] = df['MA_7'].fillna(df['MA_7'].mean())
        df['MA_30'] = df['MA_30'].fillna(df['MA_30'].mean())

        # Prepare data using MinMaxScaler
        values = df[['Average Price']].values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)

        # Create sequences
        X, y = create_dataset(scaled_values, look_back)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build model
        model = build_tat_model(input_shape=(look_back,1))

        # Train model
        history = model.fit(X, y, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

        # Save training logs
        log_file = os.path.join(output_logs, file.replace(".csv", "_tat_training.txt"))
        with open(log_file, "w") as f:
            f.write("Training Loss per Epoch:\n")
            for i, loss in enumerate(history.history['loss']):
                f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

        # Predictions
        predictions = model.predict(X)
        predictions_rescaled = scaler.inverse_transform(predictions)
        df['Predicted'] = [np.nan]*look_back + list(predictions_rescaled.flatten())
        df['Predicted'] = df['Predicted'].round(2)

        # Compute metrics safely
        y_true = df['Average Price'].values[look_back:]
        y_pred = predictions_rescaled.flatten()

        # Avoid divide-by-zero in MAPE
        non_zero_idx = y_true != 0
        if np.any(non_zero_idx):
            mape = round(np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100, 2)
            accuracy = round(100 - mape, 2)
        else:
            mape, accuracy = np.nan, np.nan

        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)

        metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

        # Save model as .pkl
        model_file = os.path.join(output_models, file.replace(".csv", "_tat_model.pkl"))
        joblib.dump(model, model_file)

        # Save CSV with only Date, Actual, Predicted
        save_df = df[['Average Price', 'Predicted']].copy()
        save_df.rename(columns={'Average Price':'Actual'}, inplace=True)
        save_df['Date'] = df['Date']
        save_df = save_df[['Date', 'Actual', 'Predicted']]
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)

        # Save graph
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue')
        plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red', linestyle='dashed')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f'Price Prediction (TAT) - {file}')
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_graph.png"))
        plt.savefig(graph_file, bbox_inches='tight')
        plt.close()

        print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%\n")

# Save all metrics
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: tomato_Bangalore_daily.csv
Epoch 1/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.0024 - mae: 0.0290 - val_loss: 0.0074 - val_mae: 0.0543
Epoch 2/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 0.0024 - mae: 0.0286 - val_loss: 0.0078 - val_mae: 0.0561
Epoch 3/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 0.0027 - mae: 0.0286 - val_loss: 0.0075 - val_mae: 0.0546
Epoch 4/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0022 - mae: 0.0282 - val_loss: 0.0068 - val_mae: 0.0523
Epoch 5/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0027 - mae: 0.0293 - val_loss: 0.0072 - val_mae: 0.0537
Epoch 6/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0025 - mae: 0.0288 - val_loss: 0.0074 - val_mae: 0.0544
Epoch 7/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0026 - mae: 0.0285 - val_loss: 0.0073 - val_mae: 0.0538
Epoch 8/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 0.0024 - mae: 0.0289 - val_loss: 0.0074 - val_ma

In [ ]:
#TAT+MQA

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import joblib  # for .pkl model saving (best-effort)
warnings_imported = False
try:
    import warnings
    warnings.filterwarnings("ignore")
    warnings_imported = True
except:
    pass

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_mqa_output_models"
output_csv = "tat_mqa_output_csv"
output_graphs = "tat_mqa_output_graphs"
output_logs = "tat_mqa_output_logs"
metrics_file = "tat_mqa_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Multi-Query Attention Layer
# -----------------------------
class MultiQueryAttention(layers.Layer):
    def __init__(self, num_heads, key_dim, dropout_rate=0.1, **kwargs):
        super(MultiQueryAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.q_dense = [layers.Dense(key_dim) for _ in range(num_heads)]
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.dropout = layers.Dropout(dropout_rate)
        self.output_dense = layers.Dense(key_dim)

    def call(self, x):
        K = self.k_dense(x)
        V = self.v_dense(x)
        head_outputs = []

        for q_layer in self.q_dense:
            Q = q_layer(x)
            scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
            attention_weights = tf.nn.softmax(scores, axis=-1)
            attention_output = tf.matmul(attention_weights, V)
            head_outputs.append(attention_output)

        concat = tf.concat(head_outputs, axis=-1)
        output = self.output_dense(concat)
        output = self.dropout(output)
        return output

    def get_config(self):
        config = super(MultiQueryAttention, self).get_config()
        config.update({
            "num_heads": self.num_heads,
            "key_dim": self.key_dim,
            "dropout_rate": self.dropout_rate
        })
        return config

# -----------------------------
# TAT-MQA Model
# -----------------------------
def build_tat_mqa_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    x = MultiQueryAttention(num_heads=num_heads, key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Function to robustly parse dates
# -----------------------------
def parse_dates_safe(date_series):
    # keep same behavior as you used before: infer_datetime_format True fallback
    try:
        return pd.to_datetime(date_series, infer_datetime_format=True, errors='coerce')
    except:
        return pd.to_datetime(date_series, errors='coerce')

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if not file.endswith(".csv"):
        continue

    file_path = os.path.join(input_folder, file)
    print(f"🚀 Processing: {file}")

    # Load CSV
    df = pd.read_csv(file_path)

    # Robust date parsing
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.dropna(subset=['Date'])
    df = df.sort_values('Date').reset_index(drop=True)

    # -----------------------------
    # Handle missing Average Price
    # -----------------------------
    if 'Average Price' not in df.columns:
        raise KeyError("Input CSV must contain 'Average Price' column.")
    if df['Average Price'].isna().sum() > 0:
        mean_val = df['Average Price'].mean()
        df['Average Price'].fillna(mean_val, inplace=True)
        print(f"Filled {df['Average Price'].isna().sum()} missing Average Price values with mean {mean_val:.2f}")

    # Round Actual values early
    df['Average Price'] = df['Average Price'].astype(float).round(2)

    # Moving averages (fill remaining NaNs with column mean to avoid plotting gaps)
    df['MA_7'] = df['Average Price'].rolling(window=7).mean()
    df['MA_30'] = df['Average Price'].rolling(window=30).mean()
    df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
    df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

    # Prepare data
    values = df[['Average Price']].values.astype('float32')
    if len(values) <= look_back:
        # Not enough data for the chosen look_back; skip file with a warning
        print(f"⚠️ Skipping {file} — dataset length ({len(values)}) <= look_back ({look_back})")
        continue

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_values = scaler.fit_transform(values)

    X, y = create_dataset(scaled_values, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Build and train model
    model = build_tat_mqa_model(input_shape=(look_back,1))
    model.summary()

    history = model.fit(X, y, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

    # Save training logs
    log_file = os.path.join(output_logs, file.replace(".csv", "_tat_mqa_training.txt"))
    with open(log_file, "w") as f:
        f.write("Training Loss per Epoch:\n")
        for i, loss in enumerate(history.history['loss']):
            val_loss = history.history['val_loss'][i] if 'val_loss' in history.history else None
            f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={val_loss}\n")

    # Predictions (rescale)
    predictions = model.predict(X)
    predictions_rescaled = scaler.inverse_transform(predictions).flatten()
    # pad with NaNs at start so predicted series aligns with original df index
    padded_preds = np.concatenate([np.full(look_back, np.nan), predictions_rescaled])
    df['Predicted'] = np.round(padded_preds, 2)

    # -----------------------------
    # Compute metrics using only the matched portion (exclude leading NaNs)
    # -----------------------------
    y_true = df['Average Price'].values[look_back:]
    y_pred = predictions_rescaled
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    # MAPE: avoid divide-by-zero by masking zeros
    mask = y_true != 0
    if mask.sum() == 0:
        mape = np.nan
        accuracy = np.nan
    else:
        mape = round(np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100, 2)
        accuracy = round(100 - mape, 2)

    metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

    print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%")

    # -----------------------------
    # Save model: native Keras and attempt .pkl (best-effort)
    # -----------------------------
    keras_model_path = os.path.join(output_models, file.replace(".csv", "_tat_mqa_model.keras"))
    model.save(keras_model_path)
    pkl_model_path = os.path.join(output_models, file.replace(".csv", "_tat_mqa_model.pkl"))
    try:
        # joblib.dump the Keras model object (may fail on some TF versions)
        joblib.dump(model, pkl_model_path)
    except Exception as e:
        # fallback: save model architecture + weights dict to pickle (best-effort)
        try:
            model_info = {
                "config": model.get_config(),
                "weights": model.get_weights()
            }
            joblib.dump(model_info, pkl_model_path)
        except Exception as e2:
            print(f"⚠️ Could not save .pkl for {file}: {e}; {e2}")

    # -----------------------------
    # Save CSV with only Date, Actual, Predicted (Actual renamed)
    # -----------------------------
    save_df = pd.DataFrame({
        'Date': df['Date'],
        'Actual': df['Average Price'].round(2),
        'Predicted': df['Predicted']
    })
    updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_mqa_updated.csv"))
    save_df.to_csv(updated_csv_path, index=False)
    print(f"Saved updated CSV with Date, Actual & Predicted: {updated_csv_path}")

    # -----------------------------
    # Save graph with Actual, Predicted, MA7, MA30
    # -----------------------------
    plt.figure(figsize=(12,6))
    plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue')
    plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red', linestyle='dashed')
    plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
    plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
    plt.xlabel('Date')
    plt.ylabel('Average Price')
    plt.title(f'Price Prediction (TAT-MQA) - {file}')
    plt.legend()
    plt.grid(True)
    graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_mqa_graph.png"))
    plt.savefig(graph_file, bbox_inches='tight')
    plt.close()

# -----------------------------
# Save all metrics
# -----------------------------
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: tomato_Bangalore_daily.csv



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_query_attention         │ (None, 30, 64)            │          17,216 │ input_layer[0][0]          │
│ (MultiQueryAttention)         │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 30, 64)            │               0 │ multi_query_attention[0][… │
│                               │                           │                 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization           │ (None, 30, 64)            │             128 │ add[0][0]                  │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_7 (Dense)               │ (None, 30, 128)           │           8,320 │ layer_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_8 (Dense)               │ (None, 30, 1)             │             129 │ dense_7[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_1 (Add)                   │ (None, 30, 64)            │               0 │ layer_normalization[0][0], │
│                               │                           │                 │ dense_8[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_1         │ (None, 30, 64)            │             128 │ add_1[0][0]                │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d      │ (None, 64)                │               0 │ layer_normalization_1[0][… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_9 (Dense)               │ (None, 1)                 │              65 │ global_average_pooling1d[… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 25,986 (101.51 KB)

 Trainable params: 25,986 (101.51 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_query_attention_18      │ (None, 30, 64)            │          17,216 │ input_layer_18[0][0]       │
│ (MultiQueryAttention)         │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_36 (Add)                  │ (None, 30, 64)            │               0 │ multi_query_attention_18[… │
│                               │                           │                 │ input_layer_18[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_36        │ (None, 30, 64)            │             128 │ add_36[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_187 (Dense)             │ (None, 30, 128)           │           8,320 │ layer_normalization_36[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_188 (Dense)             │ (None, 30, 1)             │             129 │ dense_187[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_37 (Add)                  │ (None, 30, 64)            │               0 │ layer_normalization_36[0]… │
│                               │                           │                 │ dense_188[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_37        │ (None, 30, 64)            │             128 │ add_37[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_18   │ (None, 64)                │               0 │ layer_normalization_37[0]… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_189 (Dense)             │ (None, 1)                 │              65 │ global_average_pooling1d_… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 25,986 (101.51 KB)

 Trainable params: 25,986 (101.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
328/328 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - loss: 0.0448 - mae: 0.1147 - val_loss: 0.0294 - val_mae: 0.1127
Epoch 2/100
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0071 - mae: 0.0648 - val_loss: 0.0329 - val_mae: 0.1104
Epoch 3/100
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0050 - mae: 0.0532 - val_loss: 0.0314 - val_mae: 0.1067
Epoch 4/100
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0048 - mae: 0.0521 - val_loss: 0.0254 - val_mae: 0.1129
Epoch 5/100
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0037 - mae: 0.0444 - val_loss: 0.0290 - val_mae: 0.1037
Epoch 6/100
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0034 - mae: 0.0423 - val_loss: 0.0251 - val_mae: 0.0993
Epoch 7/100
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0030 - mae: 0.0393 - val_loss: 0.0241 - val_mae: 0.0986
Epoch 8/100
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0029 - mae: 0.0391 - val_loss: 0.0213 - val_mae: 0.0931
Epoch 9/100
328/328 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
#TAT+GQA

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import joblib  # For saving models as .pkl

# -----------------------------
# Suppress Warnings
# -----------------------------
warnings.filterwarnings("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None  # Avoid SettingWithCopyWarning

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_gqa_output_models"
output_csv = "tat_gqa_output_csv"
output_graphs = "tat_gqa_output_graphs"
output_logs = "tat_gqa_output_logs"
metrics_file = "tat_gqa_metrics.csv"

for folder in [output_models, output_csv, output_graphs, output_logs]:
    os.makedirs(folder, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back, 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Grouped Query Attention Layer
# -----------------------------
class GroupedQueryAttention(layers.Layer):
    def __init__(self, num_groups=2, key_dim=64, dropout_rate=0.1, **kwargs):
        super(GroupedQueryAttention, self).__init__(**kwargs)
        self.num_groups = num_groups
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.q_dense_groups = [layers.Dense(key_dim) for _ in range(num_groups)]
        self.dropout = layers.Dropout(dropout_rate)
        self.output_dense = layers.Dense(key_dim)

    def call(self, x):
        K = self.k_dense(x)
        V = self.v_dense(x)
        group_outputs = []
        for q_layer in self.q_dense_groups:
            Q = q_layer(x)
            scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
            attention_weights = tf.nn.softmax(scores, axis=-1)
            attn_output = tf.matmul(attention_weights, V)
            group_outputs.append(attn_output)
        concat = tf.concat(group_outputs, axis=-1)
        output = self.output_dense(concat)
        output = self.dropout(output)
        return output

    def get_config(self):
        config = super(GroupedQueryAttention, self).get_config()
        config.update({
            "num_groups": self.num_groups,
            "key_dim": self.key_dim,
            "dropout_rate": self.dropout_rate
        })
        return config

# -----------------------------
# TAT-GQA Model
# -----------------------------
def build_tat_gqa_model(input_shape, d_model=64, num_groups=2, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    x = GroupedQueryAttention(num_groups=num_groups, key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Training Configuration
# -----------------------------
look_back = 30
epochs = 100
batch_size = 16

# -----------------------------
# Process each CSV file
# -----------------------------
for file in os.listdir(input_folder):
    if not file.endswith(".csv"):
        continue

    file_path = os.path.join(input_folder, file)
    print(f"🚀 Processing: {file}")

    # Load CSV
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"❌ Failed to read {file}: {e}")
        continue

    # Parse dates safely (no infer_datetime_format)
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Date']).sort_values('Date')

    # Check if column exists
    if 'Average Price' not in df.columns:
        print(f"⚠️ Skipping {file}: 'Average Price' column not found.")
        continue

    # Handle missing price values
    df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

    # Moving averages (for visualization only)
    df['MA_7'] = df['Average Price'].rolling(window=7).mean().fillna(df['Average Price'].mean())
    df['MA_30'] = df['Average Price'].rolling(window=30).mean().fillna(df['Average Price'].mean())

    # Prepare scaled data
    values = df[['Average Price']].astype('float32').values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_values = scaler.fit_transform(values)
    X, y = create_dataset(scaled_values, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Build and train model
    model = build_tat_gqa_model(input_shape=(look_back, 1))
    history = model.fit(
        X, y,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        verbose=1
    )

    # Save training log
    log_file = os.path.join(output_logs, file.replace(".csv", "_tat_gqa_training.txt"))
    with open(log_file, "w") as f:
        f.write("Epoch\tLoss\tVal_Loss\n")
        for i in range(len(history.history['loss'])):
            f.write(f"{i+1}\t{history.history['loss'][i]:.6f}\t{history.history['val_loss'][i]:.6f}\n")

    # Predictions
    predictions = model.predict(X)
    predictions_rescaled = scaler.inverse_transform(predictions)
    df['Predicted'] = [np.nan] * look_back + list(predictions_rescaled.flatten())

    # Round values for clean output
    df['Average Price'] = df['Average Price'].round(2)
    df['Predicted'] = df['Predicted'].round(2)

    # Metrics Calculation
    y_true = df['Average Price'].values[look_back:]
    y_pred = predictions_rescaled.flatten()

    # Avoid division by zero or NaN in MAPE
    mask = y_true != 0
    if np.any(mask):
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = 0.0

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 2)
    mape = round(mape, 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append([file.replace(".csv", ""), mae, rmse, r2, mape, accuracy])

    # Save model as .pkl
    model_file = os.path.join(output_models, file.replace(".csv", "_tat_gqa_model.pkl"))
    joblib.dump(model, model_file)

    # Save predictions to CSV
    save_df = df[['Date', 'Average Price', 'Predicted']].rename(columns={'Average Price': 'Actual'})
    updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_gqa_updated.csv"))
    save_df.to_csv(updated_csv_path, index=False)

    # Plot Graph
    plt.figure(figsize=(12, 6))
    plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue')
    plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
    plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
    plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT-GQA)', color='red', linestyle='dashed')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'Price Prediction (TAT-GQA) - {file}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_gqa_graph.png"))
    plt.savefig(graph_file)
    plt.close()

    print(f"✅ Done: {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%")

# -----------------------------
# Save Metrics
# -----------------------------
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 All metrics saved to: {metrics_file}")

🚀 Processing: tomato_Bangalore_daily.csv
Epoch 1/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - loss: 0.0272 - mae: 0.0799 - val_loss: 0.0023 - val_mae: 0.0302
Epoch 2/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 0.0024 - mae: 0.0308 - val_loss: 0.0024 - val_mae: 0.0400
Epoch 3/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 0.0017 - mae: 0.0241 - val_loss: 0.0023 - val_mae: 0.0337
Epoch 4/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.0017 - mae: 0.0225 - val_loss: 0.0024 - val_mae: 0.0326
Epoch 5/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - loss: 0.0016 - mae: 0.0218 - val_loss: 0.0015 - val_mae: 0.0250
Epoch 6/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - loss: 0.0016 - mae: 0.0216 - val_loss: 0.0027 - val_mae: 0.0306
Epoch 7/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - loss: 0.0015 - mae: 0.0205 - val_loss: 0.0018 - val_mae: 0.0320
Epoch 8/100
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 0.0015 - mae: 0.0199 - val_loss: 0.00

In [ ]:
#TAT+HA

In [3]:
import os
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# =========================================================
# CLEAN STARTUP – suppress all unnecessary warnings/logs
# =========================================================
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Disable TensorFlow INFO/WARNING logs
tf.get_logger().setLevel("ERROR")

# =========================================================
# Output directories
# =========================================================
input_folder = "dataset"
output_models = "tat_ha_output_models"
output_csv = "tat_ha_output_csv"
output_graphs = "tat_ha_output_graphs"
output_logs = "tat_ha_output_logs"
metrics_file = "tat_ha_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# =========================================================
# Dataset creation
# =========================================================
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back, 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)

# =========================================================
# Local Attention Layer
# =========================================================
class LocalAttention(layers.Layer):
    def __init__(self, key_dim=64, dropout_rate=0.1, **kwargs):
        super(LocalAttention, self).__init__(**kwargs)
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.q_dense = layers.Dense(key_dim)
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.dropout = layers.Dropout(dropout_rate)

    def call(self, x):
        Q = self.q_dense(x)
        K = self.k_dense(x)
        V = self.v_dense(x)
        scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(
            tf.cast(tf.shape(K)[-1], tf.float32))
        weights = tf.nn.softmax(scores, axis=-1)
        output = tf.matmul(weights, V)
        return self.dropout(output)

# =========================================================
# Hierarchical Attention Model
# =========================================================
def build_tat_ha_model(input_shape, d_model=64, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    local_attn = LocalAttention(key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    local_attn = layers.LayerNormalization(epsilon=1e-6)(local_attn + inputs)

    global_attn = layers.MultiHeadAttention(num_heads=4, key_dim=d_model)(local_attn, local_attn)
    global_attn = layers.Dropout(dropout_rate)(global_attn)
    global_attn = layers.LayerNormalization(epsilon=1e-6)(global_attn + local_attn)

    ff = layers.Dense(ff_dim, activation='relu')(global_attn)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(ff + global_attn)
    x = layers.GlobalAveragePooling1D()(x)

    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# =========================================================
# Metrics list
# =========================================================
metrics_list = []

# =========================================================
# Process each CSV file
# =========================================================
look_back = 30
for file in os.listdir(input_folder):
    if not file.endswith(".csv"):
        continue

    file_path = os.path.join(input_folder, file)
    print(f"\n🚀 Processing: {file}")

    # Load CSV
    df = pd.read_csv(file_path)

    # Safe datetime conversion
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Date']).sort_values('Date')

    # Ensure target column exists
    if 'Average Price' not in df.columns:
        print(f"⚠️ Skipping {file}: 'Average Price' missing.")
        continue

    # Replace inplace warnings (no inplace=True used)
    df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())
    df['MA_7'] = df['Average Price'].rolling(window=7).mean()
    df['MA_30'] = df['Average Price'].rolling(window=30).mean()
    df['MA_7'] = df['MA_7'].fillna(df['MA_7'].mean())
    df['MA_30'] = df['MA_30'].fillna(df['MA_30'].mean())

    # Prepare training data
    values = df[['Average Price']].values.astype('float32')
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_values = scaler.fit_transform(values)
    if len(scaled_values) <= look_back:
        print(f"⚠️ Skipping {file}: not enough data points.")
        continue

    X, y = create_dataset(scaled_values, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Build and train model (show epochs cleanly)
    model = build_tat_ha_model(input_shape=(look_back, 1))
    history = model.fit(X, y, epochs=50, batch_size=16, validation_split=0.2, verbose=2)

    # Save logs
    log_file = os.path.join(output_logs, file.replace(".csv", "_tat_ha_training.txt"))
    with open(log_file, "w") as f:
        f.write("Training Loss per Epoch:\n")
        for i, loss in enumerate(history.history['loss']):
            f.write(f"Epoch {i + 1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

    # Predictions
    predictions = model.predict(X, verbose=0)
    predictions_rescaled = scaler.inverse_transform(predictions)
    df['Predicted'] = [np.nan] * look_back + list(predictions_rescaled.flatten())

    # Round results
    df['Average Price'] = df['Average Price'].round(2)
    df['Predicted'] = df['Predicted'].round(2)

    # Evaluate
    y_true = df['Average Price'].values[look_back:]
    y_pred = predictions_rescaled.flatten()
    valid_idx = ~np.isnan(y_true) & ~np.isnan(y_pred) & (y_true != 0)

    if not np.any(valid_idx):
        print(f"⚠️ Skipping metrics for {file}: invalid or zero values.")
        continue

    y_true, y_pred = y_true[valid_idx], y_pred[valid_idx]
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 2)
    mape = round(np.mean(np.abs((y_true - y_pred) / np.maximum(y_true, 1e-10))) * 100, 2)
    accuracy = round(max(0, min(100, 100 - mape)), 2)

    metrics_list.append([file.replace(".csv", ""), mae, rmse, r2, mape, accuracy])

    # Save model
    model_file = os.path.join(output_models, file.replace(".csv", "_tat_ha_model.pkl"))
    joblib.dump(model, model_file)

    # Save updated CSV
    save_df = df[['Date', 'Average Price', 'Predicted']].rename(columns={'Average Price': 'Actual'})
    updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_ha_updated.csv"))
    save_df.to_csv(updated_csv_path, index=False)

    # Plot graph
    plt.figure(figsize=(12, 6))
    plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue')
    plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
    plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
    plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT-HA)', color='red', linestyle='dashed')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'Price Prediction (TAT-HA) - {file}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_ha_graph.png"))
    plt.savefig(graph_file)
    plt.close()

    print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%")

# Save metrics
if metrics_list:
    metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
    metrics_df.to_csv(metrics_file, index=False)
    print(f"\n📊 Metrics saved to {metrics_file}")
else:
    print("\n⚠️ No valid data found to save metrics.")

2092/2092 - 30s - 14ms/step - loss: 9.6616e-04 - mae: 0.0204 - val_loss: 0.0039 - val_mae: 0.0391
Epoch 50/50
2092/2092 - 31s - 15ms/step - loss: 9.6764e-04 - mae: 0.0204 - val_loss: 0.0039 - val_mae: 0.0395
✅ Done with tomato_Kolar_daily.csv | MAE=271.06, RMSE=446.48, R2=0.74, MAPE=25.54%, Accuracy=74.46%

🚀 Processing: tomato_MadikeriKodagu_daily.csv
Epoch 1/50
253/253 - 11s - 44ms/step - loss: 0.0721 - mae: 0.2085 - val_loss: 0.0185 - val_mae: 0.1221
Epoch 2/50
253/253 - 3s - 11ms/step - loss: 0.0495 - mae: 0.1763 - val_loss: 0.0080 - val_mae: 0.0756
Epoch 3/50
253/253 - 4s - 18ms/step - loss: 0.0462 - mae: 0.1688 - val_loss: 0.0043 - val_mae: 0.0576
Epoch 4/50
253/253 - 3s - 11ms/step - loss: 0.0465 - mae: 0.1688 - val_loss: 0.0024 - val_mae: 0.0374
Epoch 5/50
253/253 - 4s - 16ms/step - loss: 0.0443 - mae: 0.1651 - val_loss: 0.0055 - val_mae: 0.0663
Epoch 6/50
253/253 - 4s - 14ms/step - loss: 0.0440 - mae: 0.1639 - val_loss: 0.0023 - val_mae: 0.0362
Epoch 7/50
253/253 - 3s - 12ms/s

In [1]:
#LSTM

In [6]:
import os
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# -----------------------------
# 🧹 Clean console output
# -----------------------------
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel("ERROR")

# -----------------------------
# Folders
# -----------------------------
input_folder = "dataset"
output_models = "output_lstm_models"
output_csv = "output_lstm_csv"
output_graphs = "output_lstm_graphs"
output_logs = "output_lstm_logs"
metrics_file = os.path.join(output_csv, "lstm_metrics.csv")

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create sequences
# -----------------------------
def create_sequences(data, seq_length=5):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# -----------------------------
# Store metrics
# -----------------------------
metrics_list = []

# -----------------------------
# Start Processing CSVs
# -----------------------------
seq_length = 5

for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        district_name = file.split(".")[0]
        print(f"\n========== Processing: {file} ==========")

        df = pd.read_csv(os.path.join(input_folder, file))

        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Fill missing prices
        df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

        # Moving averages (for graph only)
        df['MA_7'] = df['Average Price'].rolling(7).mean().fillna(df['Average Price'].mean())
        df['MA_30'] = df['Average Price'].rolling(30).mean().fillna(df['Average Price'].mean())

        # Scaling
        scaler = MinMaxScaler()
        price_scaled = scaler.fit_transform(df['Average Price'].values.reshape(-1, 1))

        # Create sequences
        X, y = create_sequences(price_scaled, seq_length)

        # Train-test split (80-20)
        split = int(0.8 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        # Reshape to (samples, seq_length, 1)
        X_train = X_train.reshape((X_train.shape[0], seq_length, 1))
        X_test = X_test.reshape((X_test.shape[0], seq_length, 1))

        # -----------------------------
        # Build LSTM model
        # -----------------------------
        model = Sequential([
            LSTM(50, activation='relu', input_shape=(seq_length, 1)),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])

        # -----------------------------
        # Train Model
        # -----------------------------
        history = model.fit(
            X_train, y_train,
            epochs=20,
            batch_size=32,
            validation_data=(X_test, y_test),
            verbose=1
        )

        # -----------------------------
        # Predictions
        # -----------------------------
        y_pred_scaled = model.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred_scaled)
        y_true = scaler.inverse_transform(y_test.reshape(-1, 1))

        y_true_round = np.round(y_true, 2)
        y_pred_round = np.round(y_pred, 2)

        # -----------------------------
        # Save Predictions
        # -----------------------------
        df_pred = df.iloc[seq_length + split:].copy()
        df_pred = df_pred[['Date']].copy()
        df_pred['Actual'] = y_true_round.flatten()
        df_pred['Predicted'] = y_pred_round.flatten()

        df_pred.to_csv(os.path.join(output_csv, f"{district_name}_lstm_updated.csv"), index=False)

        # -----------------------------
        # Metrics
        # -----------------------------
        mae_val = round(mean_absolute_error(y_true, y_pred), 2)
        rmse_val = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)  # FIXED
        r2_val = round(r2_score(y_true, y_pred), 2)
        mape_val = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy_val = round(100 - mape_val, 2)

        metrics_list.append([district_name, mae_val, rmse_val, r2_val, mape_val, accuracy_val])

        print(f"✅ Done: {district_name} | MAE={mae_val}, RMSE={rmse_val}, R2={r2_val}, MAPE={mape_val}%, Accuracy={accuracy_val}%")

        # -----------------------------
        # Save Model
        # -----------------------------
        model_path = os.path.join(output_models, f"{district_name}_lstm_model.pkl")
        joblib.dump(model, model_path)

        # -----------------------------
        # Prediction Graph
        # -----------------------------
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['Average Price'], label='Actual Price', color='blue')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.plot(df_pred['Date'], df_pred['Predicted'], label='Predicted', color='red', linestyle='dashed')
        plt.title(f"LSTM Daily Prediction - {district_name}")
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_graphs, f"{district_name}_lstm_graph.png"))
        plt.close()

        # -----------------------------
        # Training Loss Graph
        # -----------------------------
        plt.figure(figsize=(8, 4))
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(f"Training Loss - {district_name}")
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(os.path.join(output_graphs, f"{district_name}_lstm_training_loss.png"))
        plt.close()

# -----------------------------
# Save Metrics
# -----------------------------
metrics_df = pd.DataFrame(
    metrics_list,
    columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)']
)
metrics_df.to_csv(metrics_file, index=False)

print(f"\n📊 Metrics saved to: {metrics_file}")
print("\n🎉 All districts processed successfully!")


========== Processing: tomato_Bangalore_daily.csv ==========
Epoch 1/20
692/692 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.0017 - mae: 0.0193 - val_loss: 0.0011 - val_mae: 0.0215
Epoch 2/20
692/692 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0015 - mae: 0.0170 - val_loss: 0.0012 - val_mae: 0.0250
Epoch 3/20
692/692 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0015 - mae: 0.0168 - val_loss: 0.0014 - val_mae: 0.0238
Epoch 4/20
692/692 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0015 - mae: 0.0167 - val_loss: 0.0025 - val_mae: 0.0282
Epoch 5/20
692/692 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0015 - mae: 0.0167 - val_loss: 0.0013 - val_mae: 0.0224
Epoch 6/20
692/692 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0015 - mae: 0.0167 - val_loss: 0.0016 - val_mae: 0.0240
Epoch 7/20
692/692 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0015 - mae: 0.0169 - val_loss: 0.0016 - val_mae: 0.0241
Epoch 8/20
692/692 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0015 - mae: 0.0167 - val_loss: 0.0013 - val_mae: 0

In [ ]:
#GRU

In [8]:
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

# -----------------------------
# Output paths
# -----------------------------
input_folder = "dataset"
output_models = "gru_output_models"
output_csv = "gru_output_csv"
output_graphs = "gru_output_graphs"
output_logs = "gru_output_logs"
metrics_file = os.path.join(output_csv, "gru_metrics.csv")

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back, 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        district_name = file.split(".")[0]
        print(f"\n🚀 Processing: {file}")

        df = pd.read_csv(os.path.join(input_folder, file))

        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

        df['MA_7'] = df['Average Price'].rolling(window=7).mean().fillna(df['Average Price'].mean())
        df['MA_30'] = df['Average Price'].rolling(window=30).mean().fillna(df['Average Price'].mean())

        values = df[['Average Price']].values
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)

        X, y = create_dataset(scaled_values, look_back)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        split = int(len(X) * 0.8)
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        model = Sequential([
            GRU(64, return_sequences=True, input_shape=(look_back, 1)),
            Dropout(0.2),
            GRU(32),
            Dropout(0.2),
            Dense(1)
        ])
        model.compile(optimizer="adam", loss="mse", metrics=['mae'])

        history = model.fit(
            X_train, y_train,
            epochs=20,
            batch_size=16,
            validation_data=(X_test, y_test),
            verbose=1
        )

        # Save training log
        log_file = os.path.join(output_logs, f"{district_name}_gru_training.txt")
        with open(log_file, "w") as f:
            f.write("Epoch\tTrain_Loss\tVal_Loss\n")
            for i, (loss, val_loss) in enumerate(zip(history.history['loss'], history.history['val_loss'])):
                f.write(f"{i+1}\t{loss:.6f}\t{val_loss:.6f}\n")

        # Predictions
        y_pred_scaled = model.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred_scaled)
        y_true = scaler.inverse_transform(y_test.reshape(-1, 1))

        y_true_round = np.round(y_true, 2)
        y_pred_round = np.round(y_pred, 2)

        df_pred = df.iloc[-len(y_true):].copy()
        df_pred = df_pred[['Date']].reset_index(drop=True)
        df_pred['Actual'] = y_true_round.flatten()
        df_pred['Predicted'] = y_pred_round.flatten()
        df_pred.to_csv(os.path.join(output_csv, f"{district_name}_gru_updated.csv"), index=False)

        # -----------------------------
        # FIXED METRICS (NO ERROR)
        # -----------------------------
        mae_val = round(mean_absolute_error(y_true, y_pred), 2)
        rmse_val = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)  # FIXED
        r2_val = round(r2_score(y_true, y_pred), 2)
        mape_val = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy_val = round(100 - mape_val, 2)

        metrics_list.append([district_name, mae_val, rmse_val, r2_val, mape_val, accuracy_val])

        print(
            f"✅ Done with {file} | "
            f"MAE={mae_val}, RMSE={rmse_val}, R2={r2_val}, "
            f"MAPE={mape_val}%, Accuracy={accuracy_val}%"
        )

        # Save model
        model_file = os.path.join(output_models, f"{district_name}_gru_model.pkl")
        joblib.dump(model, model_file)

        # Graphs
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['Average Price'], label="Actual", color="blue")
        plt.plot(df['Date'], df['MA_7'], label="MA_7", color="orange")
        plt.plot(df['Date'], df['MA_30'], label="MA_30", color="green")
        plt.plot(df_pred['Date'], df_pred['Predicted'], label="Predicted (GRU)", color="red", linestyle="dashed")
        plt.xlabel("Date")
        plt.ylabel("Average Price")
        plt.title(f"GRU Predictions - {district_name}")
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_graphs, f"{district_name}_gru_graph.png"))
        plt.close()

        plt.figure(figsize=(8, 4))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.title(f"GRU Training Loss - {district_name}")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(os.path.join(output_graphs, f"{district_name}_gru_loss.png"))
        plt.close()

# Save metrics CSV
metrics_df = pd.DataFrame(
    metrics_list,
    columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)']
)
metrics_df.to_csv(metrics_file, index=False)

print("\n📊 Metrics saved to", metrics_file)
print("\n✅ All districts processed successfully!")


🚀 Processing: tomato_Bangalore_daily.csv
Epoch 1/20
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 51s 27ms/step - loss: 0.0016 - mae: 0.0194 - val_loss: 0.0010 - val_mae: 0.0211
Epoch 2/20
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 35s 26ms/step - loss: 0.0015 - mae: 0.0186 - val_loss: 0.0011 - val_mae: 0.0212
Epoch 3/20
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 39s 28ms/step - loss: 0.0014 - mae: 0.0178 - val_loss: 0.0019 - val_mae: 0.0301
Epoch 4/20
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 35s 25ms/step - loss: 0.0014 - mae: 0.0176 - val_loss: 0.0011 - val_mae: 0.0218
Epoch 5/20
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 39s 28ms/step - loss: 0.0013 - mae: 0.0174 - val_loss: 9.5545e-04 - val_mae: 0.0207
Epoch 6/20
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 40s 29ms/step - loss: 0.0013 - mae: 0.0172 - val_loss: 0.0010 - val_mae: 0.0208
Epoch 7/20
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 37s 27ms/step - loss: 0.0013 - mae: 0.0172 - val_loss: 0.0012 - val_mae: 0.0225
Epoch 8/20
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 37s 27ms/step - loss: 0.0013 - mae: 0.0169 - val_loss: 0.0